## Практикум 7. Задание.

Используя библиотеки, pandas, requests, pyod:
1. Скачать, используя api.hh.ru, 500 вакансий, содержащих в названии "Разработчик", с указанием ЗП
- 1.1 Оставить только id, name, area.name, salary.from, salary.to, salary.gross, salary.currency, employer.name
2. Используя средства pandas, сохранить эти вакансии в виде df.csv, df.json.
3. Создать пустые базы db1 и db2 в Sqlite3, c полями из 1.1
4. Наполнить базы, используя, соответственно считывание из csv и из json.
5. Создать пустую базу db3. Создать три таблицы - вакансии, города, и работодатели. Нормализовать таблицу из 1.1 и разбить ее данные на 3 перечисленные таблицы.
6. Средствами SQL создать новую колонку очищенной зарплаты в db3, содержащую
среднюю зп по полям from и to, пересчитанную в рубли и очищенную от налога (для простоты - 13%)
7. По таблицам db3 с помощью JOIN, GROUP BY, HAVING, ORDER BY,
вывести по убыванию очищенной зарплаты список вакансий в городах Москва и Ташкент
8. С помощью оконных функций (dense_rank) вывести в каждом городе вакансию с максимальной очищенной зп
9. С помощью оконных функций (lag, lead, next) для каждой вакансии вывести макс и мин очищенную зп

In [ ]:
import pandas as pd
import requests
import sqlite3
import re
from typing import List, Dict

####  1. Скачать, используя api.hh.ru, 500 вакансий, содержащих в названии "Разработчик", с указанием ЗП

In [2]:
url = "https://api.hh.ru/vacancies"

data = []
for i in range(5):
    params = {
        "text": "Разработчик",
        "area": 1,
        "per_page": 100,
        "page": i,
        "only_with_salary": True,
    }
    response = requests.get(url, params=params)
    data += response.json()["items"]

In [3]:
len(data)

500

1.1 Оставить только id, name, area.name, salary.from, salary.to, salary.gross, salary.currency, employer.name

In [4]:
df = pd.DataFrame(data)
df = df[["id", "name", "area", "salary", "snippet", "employer"]]
df["area_name"] = df["area"].apply(lambda x: x["name"])
df["salary_from"] = df["salary"].apply(lambda x: x["from"])
df["salary_to"] = df["salary"].apply(lambda x: x["to"])
df["salary_gross"] = df["salary"].apply(lambda x: x["gross"])
df["salary_currency"] = df["salary"].apply(lambda x: x["currency"])
df["employer_name"] = df["employer"].apply(lambda x: x["name"])
df = df[
    [
        "id",
        "name",
        "area_name",
        "salary_from",
        "salary_to",
        "salary_gross",
        "salary_currency",
        "employer_name",
    ]
]
df

,id,name,area_name,salary_from,salary_to,salary_gross,salary_currency,employer_name
0,112564663,Junior/Middle Frontend разработчик (React),Москва,NaN,70000.0,False,RUR,Hammer Systems
1,112763765,"Junior front-end разработчик (TypeScript, React)",Москва,100000.0,180000.0,False,RUR,Mindbox
2,112783190,Инженер по охране окружающей среды,Москва,200000.0,NaN,False,RUR,LOTS.Development
3,112746981,Frontend-разработчик,Москва,70000.0,NaN,False,RUR,Aiti Guru
4,112620411,Главный бухгалтер,Москва,350000.0,500000.0,False,RUR,ФАЙВ ЛЕМОНС ГРУПП
...,...,...,...,...,...,...,...,...
495,112294699,Senior Javascript / Typescript разработчик,Москва,300000.0,400000.0,False,RUR,Айкрафт
496,112576157,Программист станков ЧПУ,Москва,200000.0,NaN,False,RUR,Центр Электронной Коммерции
497,111089779,SMM-менеджер (SMM),Москва,150000.0,NaN,False,RUR,BYTIME
498,111960819,Frontend разработчик (Vue),Москва,160000.0,NaN,False,RUR,Комс


#### 2. Используя средства pandas, сохранить эти вакансии в виде df.csv, df.json.

In [5]:
df.to_csv("data/df.csv", index=False)
df.to_json("data/df.json", orient="records", lines=True)

#### 3. Создать пустые базы db1 и db2 в Sqlite3, c полями из 1.1

In [6]:
# Create db1
conn1 = sqlite3.connect("data/db1.sqlite")
c1 = conn1.cursor()
c1.execute(f"DROP TABLE IF EXISTS vacancies")
c1.execute("""
CREATE TABLE vacancies (
    id TEXT,
    name TEXT,
    area_name TEXT,
    salary_from REAL,
    salary_to REAL,
    salary_gross BOOLEAN,
    salary_currency TEXT,
    employer_name TEXT
)
""")
conn1.commit()
conn1.close()

# Create db2
conn2 = sqlite3.connect("data/db2.sqlite")
c2 = conn2.cursor()
c2.execute(f"DROP TABLE IF EXISTS vacancies")
c2.execute("""
CREATE TABLE vacancies (
    id TEXT,
    name TEXT,
    area_name TEXT,
    salary_from REAL,
    salary_to REAL,
    salary_gross BOOLEAN,
    salary_currency TEXT,
    employer_name TEXT
)
""")
conn2.commit()
conn2.close()

#### 4. Наполнить базы, используя, соответственно считывание из csv и из json.

In [7]:
def convert_types(data: List) -> List:
    output = []
    for raw_item in data:
        if raw_item.lower() in ["true", "false"]:
            output.append(raw_item.lower() == "true")
        elif raw_item.lower() == "null" or raw_item == "":
            output.append(None)
        else:
            try:
                if "." in raw_item:
                    output.append(float(raw_item))
                else:
                    output.append(int(raw_item))
            except ValueError:
                output.append(raw_item)
    return output

In [ ]:
def find_all_indices(main_str: str, sub_str: str) -> List:
    indices = []
    start = main_str.find(sub_str)
    while start != -1:
        indices.append(start)
        start += len(sub_str)
        start = main_str.find(sub_str, start)
    return indices


def parse_line(line: str, sep: str = ",", quotechar: str = '"') -> List:
    quotes = find_all_indices(line, quotechar)
    i = 0
    while len(quotes) > i + 1:
        if quotes[i + 1] - quotes[i] == 1:
            quotes.pop(i)
            quotes.pop(i)
        else:
            i += 1

    if not quotes:
        return convert_types(line.split(sep=sep))
    else:
        if len(quotes) % 2 != 0:
            raise ValueError("Invalid quoting.")
        output = []
        quote_data = [line[: quotes[0]]]
        quote_data += [
            line[quotes[i] : quotes[i + 1] + 1] for i in range(0, len(quotes) - 1, 1)
        ]
        quote_data += [line[quotes[-1] + 1 :]]
        quote = False
        for data in quote_data:
            if quote:
                output.append(data[1:-1])
            elif data:
                raw_data = data
                if raw_data[0] == "," and not line.startswith(raw_data):
                    raw_data = raw_data[1:]
                if raw_data[-1] == "," and not line.endswith(raw_data):
                    raw_data = raw_data[:-1]

                output += convert_types(raw_data.split(sep=sep))
            quote = not quote
        return [item for item in output if item not in ["", '"']]


def csv_to_sql(
    db_path: str,
    csv_path: str,
    table_name: str,
    drop_if_exist: bool = False,
    sep: str = ",",
):
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    if drop_if_exist:
        cursor.execute(f"DROP TABLE IF EXISTS {table_name}")

    with open(csv_path, "r", encoding="utf-8") as csv_file:
        headers = csv_file.readline().split(sep=sep)

        cursor.execute(
            f"CREATE TABLE IF NOT EXISTS {table_name} ({', '.join(headers)})"
        )

        # Insert the data into the table
        for row in csv_file:
            row = row[:-1]
            data = parse_line(row, sep=sep)
            placeholders = ", ".join(["?"] * len(data))
            cursor.execute(f"INSERT INTO {table_name} VALUES ({placeholders})", data)

    conn.commit()
    conn.close()

In [9]:
csv_to_sql("data/db1.sqlite", "data/df.csv", "vacancies", drop_if_exist=True)

112564663,Junior/Middle Frontend разработчик (React),Москва,,70000.0,False,RUR,Hammer Systems
[112564663, 'Junior/Middle Frontend разработчик (React)', 'Москва', None, 70000.0, False, 'RUR', 'Hammer Systems']
112763765,"Junior front-end разработчик (TypeScript, React)",Москва,100000.0,180000.0,False,RUR,Mindbox
[112763765, 'Junior front-end разработчик (TypeScript, React)', 'Москва', 100000.0, 180000.0, False, 'RUR', 'Mindbox']
112783190,Инженер по охране окружающей среды,Москва,200000.0,,False,RUR,LOTS.Development
[112783190, 'Инженер по охране окружающей среды', 'Москва', 200000.0, None, False, 'RUR', 'LOTS.Development']
112746981,Frontend-разработчик,Москва,70000.0,,False,RUR,Aiti Guru
[112746981, 'Frontend-разработчик', 'Москва', 70000.0, None, False, 'RUR', 'Aiti Guru']
112620411,Главный бухгалтер,Москва,350000.0,500000.0,False,RUR,ФАЙВ ЛЕМОНС ГРУПП
[112620411, 'Главный бухгалтер', 'Москва', 350000.0, 500000.0, False, 'RUR', 'ФАЙВ ЛЕМОНС ГРУПП']
111747905,Специалист контакт-центра

In [10]:
conn = sqlite3.connect("data/db1.sqlite")
df = pd.read_sql_query("SELECT * FROM vacancies", conn)
conn.close()

df

,id,name,area_name,salary_from,salary_to,salary_gross,salary_currency,employer_name
0,112564663,Junior/Middle Frontend разработчик (React),Москва,NaN,70000.0,0,RUR,Hammer Systems
1,112763765,"Junior front-end разработчик (TypeScript, React)",Москва,100000.0,180000.0,0,RUR,Mindbox
2,112783190,Инженер по охране окружающей среды,Москва,200000.0,NaN,0,RUR,LOTS.Development
3,112746981,Frontend-разработчик,Москва,70000.0,NaN,0,RUR,Aiti Guru
4,112620411,Главный бухгалтер,Москва,350000.0,500000.0,0,RUR,ФАЙВ ЛЕМОНС ГРУПП
...,...,...,...,...,...,...,...,...
495,112294699,Senior Javascript / Typescript разработчик,Москва,300000.0,400000.0,0,RUR,Айкрафт
496,112576157,Программист станков ЧПУ,Москва,200000.0,NaN,0,RUR,Центр Электронной Коммерции
497,111089779,SMM-менеджер (SMM),Москва,150000.0,NaN,0,RUR,BYTIME
498,111960819,Frontend разработчик (Vue),Москва,160000.0,NaN,0,RUR,Комс


In [11]:
def parse_json_object(data: str) -> Dict:
    json_object = {}
    key_value_pattern = (
        r"\"(.*?)\"\s*:\s*(\"[^\"]*\"|\d+(\.\d+)?|true|false|null|\{|\[|\]|\})"
    )
    matches = re.finditer(key_value_pattern, data)
    for match in matches:
        token = match.group(0)
        key = token.split(":")[0].strip().strip('"')
        value = token.split(":")[1].strip().strip('"')
        value = convert_types([value])[0]
        json_object[key] = value
    return json_object


def parse_json_file(file_path: str) -> List[Dict]:
    with open(file_path, "r", encoding="utf-8") as file:
        data = file.read()

    data = data.strip()
    data = data.encode().decode("unicode_escape")
    json_objects = []
    for obj in re.finditer(r"\{.*?\}", data, re.DOTALL):
        json_objects.append(parse_json_object(obj.group(0)))
    return json_objects

In [12]:
def json_to_sql(
    db_path: str, json_file_path: str, table_name: str, drop_if_exist: bool = False
):
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    if drop_if_exist:
        cursor.execute(f"DROP TABLE IF EXISTS {table_name}")

    data_list = parse_json_file(json_file_path)

    if not data_list:
        raise ValueError("The JSON file is empty or invalid.")

    columns = ", ".join(data_list[0].keys())
    placeholders = ", ".join(["?"] * len(data_list[0]))
    cursor.execute(f"CREATE TABLE IF NOT EXISTS {table_name} ({columns})")

    for data in data_list:
        values = list(data.values())
        cursor.execute(f"INSERT INTO {table_name} VALUES ({placeholders})", values)

    conn.commit()
    conn.close()

In [13]:
json_to_sql("data/db2.sqlite", "data/df.json", "vacancies", drop_if_exist=True)

/tmp/ipykernel_412493/1686891791.py:21: DeprecationWarning: invalid escape sequence '\/'
  data = data.encode().decode("unicode_escape")


In [14]:
conn = sqlite3.connect("data/db2.sqlite")
df = pd.read_sql_query("SELECT * FROM vacancies", conn)
conn.close()

df

,id,name,area_name,salary_from,salary_to,salary_gross,salary_currency,employer_name
0,112564663,Junior\/Middle Frontend разработчик (React),Москва,NaN,70000.0,0,RUR,Hammer Systems
1,112763765,"Junior front-end разработчик (TypeScript, React)",Москва,100000.0,180000.0,0,RUR,Mindbox
2,112783190,Инженер по охране окружающей среды,Москва,200000.0,NaN,0,RUR,LOTS.Development
3,112746981,Frontend-разработчик,Москва,70000.0,NaN,0,RUR,Aiti Guru
4,112620411,Главный бухгалтер,Москва,350000.0,500000.0,0,RUR,ФАЙВ ЛЕМОНС ГРУПП
...,...,...,...,...,...,...,...,...
495,112294699,Senior Javascript \/ Typescript разработчик,Москва,300000.0,400000.0,0,RUR,Айкрафт
496,112576157,Программист станков ЧПУ,Москва,200000.0,NaN,0,RUR,Центр Электронной Коммерции
497,111089779,SMM-менеджер (SMM),Москва,150000.0,NaN,0,RUR,BYTIME
498,111960819,Frontend разработчик (Vue),Москва,160000.0,NaN,0,RUR,Комс


#### 5. Создать пустую базу db3. Создать три таблицы - вакансии, города, и работодатели. Нормализовать таблицу из 1.1 и разбить ее данные на 3 перечисленные таблицы.

In [15]:
conn3 = sqlite3.connect("data/db3.sqlite")
c3 = conn3.cursor()

c3.execute(f"DROP TABLE IF EXISTS cities")
c3.execute(f"DROP TABLE IF EXISTS employers")
c3.execute(f"DROP TABLE IF EXISTS vacancies")

c3.execute("""
CREATE TABLE cities (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT UNIQUE
)
""")

c3.execute("""
CREATE TABLE employers (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT UNIQUE
)
""")

c3.execute("""
CREATE TABLE vacancies (
    id TEXT,
    name TEXT,
    city_id INTEGER,
    salary_from REAL,
    salary_to REAL,
    salary_gross BOOLEAN,
    salary_currency TEXT,
    employer_id INTEGER,
    FOREIGN KEY (city_id) REFERENCES cities (id),
    FOREIGN KEY (employer_id) REFERENCES employers (id)
)
""")

cities = df[["area_name"]].drop_duplicates().reset_index(drop=True)
cities["name"] = cities["area_name"]
cities = cities.drop(columns=["area_name"])
cities.to_sql("cities", conn3, if_exists="append", index=False)

employers = df[["employer_name"]].drop_duplicates().reset_index(drop=True)
employers["name"] = employers["employer_name"]
employers = employers.drop(columns=["employer_name"])
employers.to_sql("employers", conn3, if_exists="append", index=False)

df["city_id"] = df["area_name"].apply(
    lambda x: c3.execute("SELECT id FROM cities WHERE name = ?", (x,)).fetchone()[0]
)
df["employer_id"] = df["employer_name"].apply(
    lambda x: c3.execute("SELECT id FROM employers WHERE name = ?", (x,)).fetchone()[0]
)

vacancies = df[
    [
        "id",
        "name",
        "city_id",
        "salary_from",
        "salary_to",
        "salary_gross",
        "salary_currency",
        "employer_id",
    ]
]
vacancies.to_sql("vacancies", conn3, if_exists="append", index=False)

conn3.commit()
conn3.close()

#### 6. Средствами SQL создать новую колонку очищенной зарплаты в db3, содержащую среднюю зп по полям from и to, пересчитанную в рубли и очищенную от налога (для простоты - 13%)

In [18]:
conn3 = sqlite3.connect("data/db3.sqlite")
c3 = conn3.cursor()

c3.execute("""
ALTER TABLE vacancies ADD COLUMN real_salary REAL
""")

c3.execute("""
UPDATE vacancies
SET real_salary = (
    CASE
        WHEN salary_currency = 'USD' THEN ((COALESCE(salary_from, 0) + COALESCE(salary_to, 0)) / 2) * 100 * 0.87
        WHEN salary_currency = 'EUR' THEN ((COALESCE(salary_from, 0) + COALESCE(salary_to, 0)) / 2) * 105 * 0.87
        WHEN salary_currency = 'KZT' THEN ((COALESCE(salary_from, 0) + COALESCE(salary_to, 0)) / 2) * 0.2 * 0.87
        ELSE ((COALESCE(salary_from, 0) + COALESCE(salary_to, 0)) / 2) * 0.87
    END
)
""")

conn3.commit()
conn3.close()

#### 7. По таблицам db3 с помощью JOIN, GROUP BY, HAVING, ORDER BY, вывести по убыванию очищенной зарплаты список вакансий в городах Москва и Ташкент

In [19]:
conn3 = sqlite3.connect("data/db3.sqlite")
c3 = conn3.cursor()

query = """
SELECT vacancies.id, vacancies.name, cities.name as city_name, vacancies.real_salary
FROM vacancies
JOIN cities ON vacancies.city_id = cities.id
WHERE cities.name IN ('Москва', 'Ташкент')
GROUP BY vacancies.id, vacancies.name, cities.name, vacancies.real_salary
HAVING vacancies.real_salary IS NOT NULL
ORDER BY vacancies.real_salary DESC
"""

result = pd.read_sql_query(query, conn3)
conn3.close()

result

,id,name,city_name,real_salary
0,111939254,TeamLead \/ TeachLead Руководитель отдела разр...,Москва,870000.0
1,112338300,Software Engineer Blockchain,Москва,609000.0
2,112442788,Rust Developer (blockchain),Москва,587250.0
3,112650446,Lead Frontend Developer,Москва,565500.0
4,112281223,ML engineer (middle-senior),Москва,522000.0
...,...,...,...,...
491,112185460,Junior\/Junior+php программист (стажёр),Москва,13050.0
492,112355107,Программист-разработчик (Junior),Москва,13050.0
493,112712257,SMM-менеджер \/ Таргетолог,Москва,13050.0
494,112716298,Помощник администратора,Москва,13050.0


#### 8. С помощью оконных функций (dense_rank) вывести в каждом городе вакансию с максимальной очищенной зп

In [21]:
conn3 = sqlite3.connect("data/db3.sqlite")
c3 = conn3.cursor()

query = """
SELECT id, name, city_name, real_salary
FROM (
    SELECT vacancies.id, vacancies.name, cities.name as city_name, vacancies.real_salary,
           DENSE_RANK() OVER (PARTITION BY cities.name ORDER BY vacancies.real_salary DESC) as rank
    FROM vacancies
    JOIN cities ON vacancies.city_id = cities.id
) subquery
WHERE rank = 1
"""

result = pd.read_sql_query(query, conn3)
conn3.close()

result

,id,name,city_name,real_salary
0,112162117,UX\/UI-дизайнер IT-интерфейсов,Зеленоград,65250.0
1,111939254,TeamLead \/ TeachLead Руководитель отдела разр...,Москва,870000.0
2,112567699,Программист \/ Разработчик .NET (C#),Оренбург,139200.0
3,112436052,Младший тестировщик игр,Санкт-Петербург,69600.0


#### 9. С помощью оконных функций (lag, lead, next) для каждой вакансии вывести макс и мин очищенную зп

In [22]:
conn3 = sqlite3.connect("data/db3.sqlite")
c3 = conn3.cursor()

query = """
SELECT id, name, city_name, real_salary, 
    MAX(real_salary) OVER (PARTITION BY city_name) as max_salary,
    MIN(real_salary) OVER (PARTITION BY city_name) as min_salary,
    LAG(real_salary) OVER (PARTITION BY city_name ORDER BY real_salary) as prev_salary,
    LEAD(real_salary) OVER (PARTITION BY city_name ORDER BY real_salary) as next_salary
FROM (
    SELECT vacancies.id, vacancies.name, cities.name as city_name, vacancies.real_salary
    FROM vacancies
    JOIN cities ON vacancies.city_id = cities.id
) subquery
"""

result = pd.read_sql_query(query, conn3)
conn3.close()

result

,id,name,city_name,real_salary,max_salary,min_salary,prev_salary,next_salary
0,112162117,UX\/UI-дизайнер IT-интерфейсов,Зеленоград,65250.0,65250.0,65250.0,NaN,NaN
1,112698776,Unity Developer (Junior+). Разработчик игр на ...,Москва,8700.0,870000.0,8700.0,NaN,13050.0
2,112355107,Программист-разработчик (Junior),Москва,13050.0,870000.0,8700.0,8700.0,13050.0
3,111971156,Младший PR-специалист,Москва,13050.0,870000.0,8700.0,13050.0,13050.0
4,112716298,Помощник администратора,Москва,13050.0,870000.0,8700.0,13050.0,13050.0
...,...,...,...,...,...,...,...,...
495,112338300,Software Engineer Blockchain,Москва,609000.0,870000.0,8700.0,587250.0,870000.0
496,111939254,TeamLead \/ TeachLead Руководитель отдела разр...,Москва,870000.0,870000.0,8700.0,609000.0,NaN
497,112567699,Программист \/ Разработчик .NET (C#),Оренбург,139200.0,139200.0,139200.0,NaN,NaN
498,110758353,Младший программист,Санкт-Петербург,34800.0,69600.0,34800.0,NaN,69600.0
